In [1]:
import torch
import torch.nn.functional as F
from torch import nn


# 自定义层
class CenteredLayer(nn.Module):
    """构造一个没有任何参数的自定义层"""

    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x - x.mean()  # 均值向0偏移


layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [2]:
# 将层作为组件合并到构建更复杂的模型中
l1 = nn.Linear(4, 7)  # 默认初始化w和b为均匀分布~U(-根号1/4,根号1/4)
l2 = CenteredLayer()
net = nn.Sequential(l1, l2)
Y = net(torch.rand(3, 4))  # 4*8
Y, Y.mean()

(tensor([[-0.1072,  0.5934, -0.2699, -0.4587, -0.0610, -0.2429,  0.2248],
         [-0.2037,  0.8679, -0.4791,  0.0935, -0.6336,  0.1474,  0.5953],
         [ 0.0103,  0.5789, -0.4474,  0.0106, -0.5588, -0.0832,  0.4232]],
        grad_fn=<SubBackward0>),
 tensor(1.2772e-08, grad_fn=<MeanBackward0>))

In [3]:
# 自定义带参数的层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)


dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.4309,  0.5857, -0.6292],
        [-0.4336,  0.9625,  0.0839],
        [ 0.6444, -0.0238, -0.2021],
        [ 0.5703,  1.6823, -0.7473],
        [-1.0489,  1.8156, -1.6762]], requires_grad=True)

In [4]:
# 使用自定义层直接执行forward
dense(torch.rand(2, 5))

tensor([[0.8752, 2.7566, 0.0000],
        [1.0333, 1.8092, 0.0000]])

In [5]:
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])